In [1]:
import os
import glob
import random
import numpy as np
import torch
import torch.nn as nn
import nibabel as nib
from torch.utils.data import Dataset, DataLoader
from torch.nn import CrossEntropyLoss
from tqdm import tqdm
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
from skimage.filters import threshold_otsu
from scipy import ndimage
from torch.amp import autocast, GradScaler
from datetime import datetime

# ---------------------------------------
# CUDA ve CUDNN Ayarları
# ---------------------------------------
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.deterministic = False
print(f"Using device: {DEVICE}")

# ---------------------------------------
# Veri Yükleme ve Ön İşleme
# ---------------------------------------
DATA_DIR = '/kaggle/input/adniprocessed'
classes = {
    'CN_Combined': 0,
    'MCI_Combined': 1,
    'AD_Combined': 2
}

# Veri yükleme ve filtreleme
all_mgz = glob.glob(os.path.join(DATA_DIR, '**', '*.mgz'), recursive=True)
orig_files = [p for p in all_mgz if os.path.basename(p).lower() == 'orig.mgz']

valid_paths, valid_labels = [], []
for orig_path in orig_files:
    group_name = None
    cur_dir = os.path.dirname(orig_path)
    while True:
        cur_dir = os.path.dirname(cur_dir)
        if os.path.basename(cur_dir) in classes:
            group_name = os.path.basename(cur_dir)
            break
        if os.path.abspath(cur_dir) == os.path.abspath(DATA_DIR):
            break

    if group_name is None or os.path.getsize(orig_path) == 0:
        continue

    valid_paths.append(orig_path)
    valid_labels.append(classes[group_name])

# Her sınıftan 300 örnek seç
random.seed(42)
limited_paths, limited_labels = [], []
for cls_name, cls_idx in classes.items():
    cls_indices = [i for i, lab in enumerate(valid_labels) if lab == cls_idx]
    selected = random.sample(cls_indices, min(len(cls_indices), 300))
    for idx in selected:
        limited_paths.append(valid_paths[idx])
        limited_labels.append(cls_idx)

# Train/Val/Test split
paths_trainval, paths_test, labels_trainval, labels_test = train_test_split(
    limited_paths, limited_labels, test_size=0.10, stratify=limited_labels, random_state=42
)

# ---------------------------------------
# Eğitim Parametreleri
# ---------------------------------------
K = 2  # 2 fold
num_epochs = 20  # Her fold için 20 epoch
num_workers = 4
PATCH_SIZE = 112

# Sonuçları kaydetmek için klasör oluştur
results_dir = f"results_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
os.makedirs(results_dir, exist_ok=True)

# ---------------------------------------
# Dataset Sınıfı
# ---------------------------------------
class ADNI_Dataset(Dataset):
    def __init__(self, paths, labels, transform=None):
        self.paths = paths
        self.labels = labels
        self.transform = transform
        self.cache = {}
        self.max_cache_size = 100

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        try:
            if idx in self.cache:
                return self.cache[idx]

            full_path = self.paths[idx]
            img_nib = nib.load(full_path)
            img = img_nib.get_fdata().astype(np.float32)

            # Normalize
            p1, p99 = np.percentile(img, (1, 99))
            img = np.clip(img, p1, p99)
            img = (img - p1) / (p99 - p1 + 1e-6)

            # Brain mask
            thresh = threshold_otsu(img)
            brain_mask = img > thresh
            brain_mask = ndimage.binary_closing(brain_mask, structure=np.ones((3, 3, 3)))
            brain_mask = ndimage.binary_fill_holes(brain_mask)
            brain_mask = ndimage.binary_dilation(brain_mask, structure=np.ones((3, 3, 3)))

            # Largest connected component
            labeled, num_features = ndimage.label(brain_mask)
            if num_features > 1:
                counts = np.bincount(labeled.ravel())
                counts[0] = 0
                largest_label = counts.argmax()
                brain_mask = (labeled == largest_label)

            # Bounding box
            coords = np.array(np.where(brain_mask))
            if len(coords) == 0 or coords.size == 0:
                raise ValueError("Empty coordinate array")
            
            z_min, y_min, x_min = coords.min(axis=1)
            z_max, y_max, x_max = coords.max(axis=1)

            # Center crop
            center_d = (z_min + z_max) // 2
            center_h = (y_min + y_max) // 2
            center_w = (x_min + x_max) // 2

            half = PATCH_SIZE // 2
            start_d = max(0, center_d - half)
            start_h = max(0, center_h - half)
            start_w = max(0, center_w - half)

            d, h, w = img.shape
            end_d = min(d, start_d + PATCH_SIZE)
            end_h = min(h, start_h + PATCH_SIZE)
            end_w = min(w, start_w + PATCH_SIZE)

            # Adjust sizes
            if end_d - start_d < PATCH_SIZE:
                start_d = max(0, end_d - PATCH_SIZE)
            if end_h - start_h < PATCH_SIZE:
                start_h = max(0, end_h - PATCH_SIZE)
            if end_w - start_w < PATCH_SIZE:
                start_w = max(0, end_w - PATCH_SIZE)

            patch = img[start_d:end_d, start_h:end_h, start_w:end_w]

            # Padding
            if patch.shape != (PATCH_SIZE, PATCH_SIZE, PATCH_SIZE):
                padded = np.zeros((PATCH_SIZE, PATCH_SIZE, PATCH_SIZE), dtype=np.float32)
                pd_, ph_, pw_ = patch.shape
                sd = (PATCH_SIZE - pd_) // 2
                sh = (PATCH_SIZE - ph_) // 2
                sw = (PATCH_SIZE - pw_) // 2
                padded[sd:sd+pd_, sh:sh+ph_, sw:sw+pw_] = patch
                patch = padded

            # Z-score normalize
            mean = patch.mean()
            std = patch.std() + 1e-6
            patch = (patch - mean) / std

            img_tensor = torch.from_numpy(patch).unsqueeze(0)
            if self.transform:
                img_tensor = self.transform(img_tensor)

            label = torch.tensor(self.labels[idx], dtype=torch.long)

            # Cache
            if len(self.cache) >= self.max_cache_size:
                self.cache.pop(next(iter(self.cache)))
            self.cache[idx] = (img_tensor, label)

            return img_tensor, label

        except Exception as e:
            print(f"Error processing {self.paths[idx]}: {str(e)}")
            return torch.zeros((1, PATCH_SIZE, PATCH_SIZE, PATCH_SIZE)), torch.tensor(self.labels[idx], dtype=torch.long)

# ---------------------------------------
# Model Sınıfları
# ---------------------------------------
class SEBlock(nn.Module):
    def __init__(self, channels, reduction=16):
        super().__init__()
        self.fc1 = nn.Linear(channels, channels // reduction, bias=False)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Linear(channels // reduction, channels, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        b, c, d, h, w = x.size()
        y = x.view(b, c, -1).mean(dim=2)
        y = self.relu(self.fc1(y))
        y = self.sigmoid(self.fc2(y))
        y = y.view(b, c, 1, 1, 1)
        return x * y

class Inception3D_SE(nn.Module):
    def __init__(self, in_channels, ch1x1, ch3x3reduce, ch3x3, ch5x5reduce, ch5x5, pool_proj):
        super().__init__()
        self.branch1 = nn.Sequential(
            nn.Conv3d(in_channels, ch1x1, kernel_size=1, bias=False),
            nn.InstanceNorm3d(ch1x1, affine=True),
            nn.ReLU(inplace=True)
        )
        self.branch3 = nn.Sequential(
            nn.Conv3d(in_channels, ch3x3reduce, kernel_size=1, bias=False),
            nn.InstanceNorm3d(ch3x3reduce, affine=True),
            nn.ReLU(inplace=True),
            nn.Conv3d(ch3x3reduce, ch3x3, kernel_size=3, padding=1, bias=False),
            nn.InstanceNorm3d(ch3x3, affine=True),
            nn.ReLU(inplace=True)
        )
        self.branch5 = nn.Sequential(
            nn.Conv3d(in_channels, ch5x5reduce, kernel_size=1, bias=False),
            nn.InstanceNorm3d(ch5x5reduce, affine=True),
            nn.ReLU(inplace=True),
            nn.Conv3d(ch5x5reduce, ch5x5, kernel_size=5, padding=2, bias=False),
            nn.InstanceNorm3d(ch5x5, affine=True),
            nn.ReLU(inplace=True)
        )
        self.branch_pool = nn.Sequential(
            nn.MaxPool3d(kernel_size=3, stride=1, padding=1),
            nn.Conv3d(in_channels, pool_proj, kernel_size=1, bias=False),
            nn.InstanceNorm3d(pool_proj, affine=True),
            nn.ReLU(inplace=True)
        )
        out_channels = ch1x1 + ch3x3 + ch5x5 + pool_proj
        self.se = SEBlock(out_channels, reduction=16)

    def forward(self, x):
        b1 = self.branch1(x)
        b3 = self.branch3(x)
        b5 = self.branch5(x)
        bp = self.branch_pool(x)
        out = torch.cat([b1, b3, b5, bp], dim=1)
        out = self.se(out)
        return out

class Inception3DNet_SE(nn.Module):
    def __init__(self, num_classes=3, dropout=0.4):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv3d(1, 64, kernel_size=7, stride=2, padding=3, bias=False),
            nn.InstanceNorm3d(64, affine=True),
            nn.ReLU(inplace=True),
            nn.MaxPool3d(kernel_size=3, stride=2, padding=1)
        )
        self.conv2 = nn.Sequential(
            nn.Conv3d(64, 64, kernel_size=1, bias=False),
            nn.InstanceNorm3d(64, affine=True),
            nn.ReLU(inplace=True),
            nn.Conv3d(64, 192, kernel_size=3, padding=1, bias=False),
            nn.InstanceNorm3d(192, affine=True),
            nn.ReLU(inplace=True),
            nn.MaxPool3d(kernel_size=3, stride=2, padding=1)
        )
        self.inception3a = Inception3D_SE(192, 64, 96, 128, 16, 32, 32)
        self.inception3b = Inception3D_SE(256, 128, 128, 192, 32, 96, 64)
        self.maxpool3 = nn.MaxPool3d(kernel_size=3, stride=2, padding=1)
        self.inception3c = Inception3D_SE(480, 192, 96, 208, 16, 48, 64)
        self.inception3d = Inception3D_SE(512, 160, 112, 224, 24, 64, 64)
        self.inception3e = Inception3D_SE(512, 128, 128, 256, 24, 64, 64)
        self.maxpool4 = nn.MaxPool3d(kernel_size=3, stride=2, padding=1)
        self.avgpool = nn.AdaptiveAvgPool3d((1,1,1))
        self.dropout = nn.Dropout3d(p=dropout)
        self.fc = nn.Linear(512, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv3d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.InstanceNorm3d, nn.GroupNorm)):
                if hasattr(m, 'weight') and m.weight is not None:
                    nn.init.constant_(m.weight, 1)
                if hasattr(m, 'bias') and m.bias is not None:
                    nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.inception3a(x)
        x = self.inception3b(x)
        x = self.maxpool3(x)
        x = self.inception3c(x)
        x = self.inception3d(x)
        x = self.inception3e(x)
        x = self.maxpool4(x)
        x = self.avgpool(x)
        x = x.flatten(1)
        x = self.dropout(x)
        logits = self.fc(x)
        return logits

# ---------------------------------------
# Eğitim Fonksiyonları
# ---------------------------------------
def train_fold(fold, paths_train, labels_train, paths_val, labels_val, hyperparams, fold_results):
    print(f"\n===== Fold {fold}/{K} =====")
    print(f"Hyperparameters: {hyperparams}")
    
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    train_ds = ADNI_Dataset(paths_train, labels_train, transform=None)
    val_ds = ADNI_Dataset(paths_val, labels_val, transform=None)
    
    train_loader = DataLoader(
        train_ds,
        batch_size=hyperparams['batch_size'],
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True,
        prefetch_factor=2,
        persistent_workers=True,
    )
    val_loader = DataLoader(
        val_ds,
        batch_size=hyperparams['batch_size'],
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True,
        prefetch_factor=2,
        persistent_workers=True,
    )
    
    model = Inception3DNet_SE(num_classes=3, dropout=hyperparams['dropout']).to(DEVICE)
    cls_loss_fn = CrossEntropyLoss()
    optimizer = torch.optim.AdamW(
        model.parameters(), 
        lr=hyperparams['lr'], 
        weight_decay=hyperparams['weight_decay']
    )
    use_amp = (DEVICE.type == 'cuda')
    scaler = GradScaler(enabled=use_amp)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        mode='min',
        factor=0.5,
        patience=3,
        verbose=True,
        min_lr=1e-6
    )
    
    best_val_acc = 0.0
    best_model_path = os.path.join(results_dir, f'best_model_fold{fold}.pth')
    fold_history = {
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': [],
        'learning_rates': []
    }
    
    for epoch in range(1, num_epochs + 1):
        # Training
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        for imgs, labels in train_loader:
            try:
                imgs = imgs.to(DEVICE, non_blocking=True)
                labels = labels.to(DEVICE, non_blocking=True)
                optimizer.zero_grad()
                
                with autocast(device_type=DEVICE.type, enabled=use_amp):
                    logits = model(imgs)
                    loss = cls_loss_fn(logits, labels)
                
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
                
                train_loss += loss.item()
                preds = logits.argmax(dim=1)
                train_correct += (preds == labels).sum().item()
                train_total += labels.size(0)
                
            except RuntimeError as e:
                if "out of memory" in str(e):
                    if torch.cuda.is_available():
                        torch.cuda.empty_cache()
                    print(f"GPU bellek hatası: {str(e)}")
                    continue
                else:
                    raise e
        
        avg_train_loss = train_loss / len(train_loader)
        train_acc = train_correct / train_total
        
        # Validation
        model.eval()
        val_loss = 0.0
        correct_val = 0
        total_val = 0
        all_preds = []
        all_labels = []
        
        with torch.no_grad():
            for imgs, labels in val_loader:
                try:
                    imgs = imgs.to(DEVICE)
                    labels = labels.to(DEVICE)
                    with autocast(device_type=DEVICE.type, enabled=use_amp):
                        logits = model(imgs)
                        loss = cls_loss_fn(logits, labels)
                    
                    val_loss += loss.item()
                    preds_val = logits.argmax(dim=1)
                    correct_val += (preds_val == labels).sum().item()
                    total_val += labels.size(0)
                    
                    all_preds.extend(preds_val.cpu().numpy())
                    all_labels.extend(labels.cpu().numpy())
                    
                except RuntimeError as e:
                    if "out of memory" in str(e):
                        if torch.cuda.is_available():
                            torch.cuda.empty_cache()
                        print(f"GPU bellek hatası: {str(e)}")
                        continue
                    else:
                        raise e
        
        val_acc = correct_val / total_val
        avg_val_loss = val_loss / len(val_loader)
        
        # Metrikleri kaydet
        fold_history['train_loss'].append(avg_train_loss)
        fold_history['train_acc'].append(train_acc)
        fold_history['val_loss'].append(avg_val_loss)
        fold_history['val_acc'].append(val_acc)
        fold_history['learning_rates'].append(optimizer.param_groups[0]['lr'])
        
        print(f"Epoch {epoch}/{num_epochs} - "
              f"Train Loss: {avg_train_loss:.4f}, Train Acc: {train_acc:.4f}, "
              f"Val Loss: {avg_val_loss:.4f}, Val Acc: {val_acc:.4f}")
        
        scheduler.step(avg_val_loss)
        
        # En iyi modeli kaydet
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_acc': val_acc,
                'hyperparams': hyperparams
            }, best_model_path)
            
            # Confusion matrix ve classification report
            cm = confusion_matrix(all_labels, all_preds)
            report = classification_report(all_labels, all_preds, target_names=["CN","MCI","AD"], output_dict=True)
            
            # Sonuçları kaydet
            fold_results[fold] = {
                'best_val_acc': best_val_acc,
                'confusion_matrix': cm,
                'classification_report': report,
                'history': fold_history,
                'hyperparams': hyperparams,
                'true': all_labels,
                'predicted': all_preds
            }
            
            # Confusion matrix'i görselleştir ve kaydet
            plt.figure(figsize=(8,6))
            sns.heatmap(cm, annot=True, fmt='d', xticklabels=["CN","MCI","AD"], yticklabels=["CN","MCI","AD"])
            plt.title(f"Fold {fold} Confusion Matrix (Best Val Acc: {best_val_acc:.4f})")
            plt.xlabel("Predicted")
            plt.ylabel("True")
            plt.savefig(os.path.join(results_dir, f'confusion_matrix_fold{fold}.png'))
            plt.close()
    
    return fold_results

def optimize_hyperparams(fold_results):
    """Fold sonuçlarına göre hiperparametreleri optimize et"""
    if not fold_results:
        return {
            'lr': 1e-4,
            'batch_size': 4,
            'weight_decay': 1e-4,
            'dropout': 0.3
        }
    
    prev_fold = list(fold_results.keys())[-1]
    prev_results = fold_results[prev_fold]
    val_acc = prev_results['best_val_acc']
    prev_params = prev_results['hyperparams']
    
    new_params = prev_params.copy()
    
    if val_acc < 0.7:
        new_params['lr'] *= 1.5
        new_params['dropout'] = min(0.5, new_params['dropout'] + 0.1)
    elif val_acc > 0.85:
        new_params['lr'] *= 0.8
        new_params['weight_decay'] *= 1.2
    
    if val_acc < 0.75:
        new_params['batch_size'] = max(2, new_params['batch_size'] - 2)
    else:
        new_params['batch_size'] = min(8, new_params['batch_size'] + 2)
    
    print(f"\nHyperparameter optimization:")
    print(f"Previous fold val_acc: {val_acc:.4f}")
    print("Previous params:", prev_params)
    print("New params:", new_params)
    
    return new_params

# ---------------------------------------
# Ana Eğitim Döngüsü
# ---------------------------------------
print("\nStarting training...")
skf = StratifiedKFold(n_splits=K, shuffle=True, random_state=42)
fold_results = {}
current_hyperparams = None

for fold, (train_idx, val_idx) in enumerate(skf.split(paths_trainval, labels_trainval), 1):
    paths_train = [paths_trainval[i] for i in train_idx]
    labels_train = [labels_trainval[i] for i in train_idx]
    paths_val = [paths_trainval[i] for i in val_idx]
    labels_val = [labels_trainval[i] for i in val_idx]
    
    current_hyperparams = optimize_hyperparams(fold_results)
    fold_results = train_fold(fold, paths_train, labels_train, paths_val, labels_val, 
                            current_hyperparams, fold_results)
    
    with open(os.path.join(results_dir, f'fold{fold}_results.txt'), 'w') as f:
        f.write(f"Fold {fold} Results:\n")
        f.write(f"Best Validation Accuracy: {fold_results[fold]['best_val_acc']:.4f}\n")
        f.write(f"Hyperparameters: {fold_results[fold]['hyperparams']}\n\n")
        f.write("Classification Report:\n")
        f.write(classification_report(
            fold_results[fold]['true'],
            fold_results[fold]['predicted'],
            target_names=["CN","MCI","AD"]
        ))

# Final sonuçları
print("\n===== Final Results =====")
for fold in fold_results:
    print(f"\nFold {fold}:")
    print(f"Best Validation Accuracy: {fold_results[fold]['best_val_acc']:.4f}")
    print("Hyperparameters:", fold_results[fold]['hyperparams'])
    print("\nClassification Report:")
    print(classification_report(
        fold_results[fold]['true'],
        fold_results[fold]['predicted'],
        target_names=["CN","MCI","AD"]
    ))

# Sonuçları görselleştir
plt.figure(figsize=(12, 4))

# Training ve validation loss
plt.subplot(1, 2, 1)
for fold in fold_results:
    plt.plot(fold_results[fold]['history']['train_loss'], label=f'Train (Fold {fold})')
    plt.plot(fold_results[fold]['history']['val_loss'], label=f'Val (Fold {fold})')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Training ve validation accuracy
plt.subplot(1, 2, 2)
for fold in fold_results:
    plt.plot(fold_results[fold]['history']['train_acc'], label=f'Train (Fold {fold})')
    plt.plot(fold_results[fold]['history']['val_acc'], label=f'Val (Fold {fold})')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.savefig(os.path.join(results_dir, 'training_history.png'))
plt.close()

print(f"\nResults saved in directory: {results_dir}")

Using device: cuda

Starting training...

===== Fold 1/2 =====
Hyperparameters: {'lr': 0.0001, 'batch_size': 4, 'weight_decay': 0.0001, 'dropout': 0.3}


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/functional.py:1597: UserWarning: dropout3d: Received a 2-D input to dropout3d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout3d exists to provide channel-wise dropout on inputs with 3 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 4D or 5D inputs).
  warnings.warn(warn_msg)


Epoch 1/20 - Train Loss: 1.1160, Train Acc: 0.3358, Val Loss: 1.1011, Val Acc: 0.3333


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/torch/nn/functional.py:1

Epoch 2/20 - Train Loss: 1.0878, Train Acc: 0.4074, Val Loss: 1.0629, Val Acc: 0.4296


...

Epoch 10/20 - Train Loss: 0.3820, Train Acc: 0.8123, Val Loss: 0.8335, Val Acc: 0.6370
Epoch 11/20 - Train Loss: 0.4868, Train Acc: 0.7506, Val Loss: 0.9230, Val Acc: 0.5951
Epoch 12/20 - Train Loss: 0.4674, Train Acc: 0.7432, Val Loss: 0.8890, Val Acc: 0.6296
Epoch 13/20 - Train Loss: 0.4081, Train Acc: 0.7802, Val Loss: 0.9636, Val Acc: 0.5728
Epoch 14/20 - Train Loss: 0.3891, Train Acc: 0.7704, Val Loss: 1.0583, Val Acc: 0.5877
Epoch 15/20 - Train Loss: 0.3449, Train Acc: 0.7975, Val Loss: 0.7685, Val Acc: 0.6938


/usr/local/lib/python3.11/dist-packages/torch/nn/functional.py:1597: UserWarning: dropout3d: Received a 2-D input to dropout3d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout3d exists to provide channel-wise dropout on inputs with 3 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 4D or 5D inputs).
  warnings.warn(warn_msg)


Epoch 16/20 - Train Loss: 0.3509, Train Acc: 0.7753, Val Loss: 0.7678, Val Acc: 0.6914
Epoch 17/20 - Train Loss: 0.3527, Train Acc: 0.7827, Val Loss: 0.7669, Val Acc: 0.7037


/usr/local/lib/python3.11/dist-packages/torch/nn/functional.py:1597: UserWarning: dropout3d: Received a 2-D input to dropout3d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout3d exists to provide channel-wise dropout on inputs with 3 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 4D or 5D inputs).
  warnings.warn(warn_msg)


Epoch 18/20 - Train Loss: 0.3094, Train Acc: 0.8000, Val Loss: 0.7757, Val Acc: 0.6938
Epoch 19/20 - Train Loss: 0.2976, Train Acc: 0.8272, Val Loss: 0.7779, Val Acc: 0.6914
Epoch 20/20 - Train Loss: 0.2865, Train Acc: 0.8222, Val Loss: 0.7775, Val Acc: 0.6963

===== Final Results =====

Fold 1:
Best Validation Accuracy: 0.7358
Hyperparameters: {'lr': 0.0001, 'batch_size': 4, 'weight_decay': 0.0001, 'dropout': 0.3}

Classification Report:
              precision    recall  f1-score   support

          CN       0.82      0.72      0.76       135
         MCI       0.66      0.64      0.65       135
          AD       0.74      0.84      0.79       135

    accuracy                           0.74       405
   macro avg       0.74      0.74      0.73       405
weighted avg       0.74      0.74      0.73       405


Fold 2:
Best Validation Accuracy: 0.7037
Hyperparameters: {'lr': 0.0001, 'batch_size': 2, 'weight_decay': 0.0001, 'dropout': 0.3}

Classification Report:
              precisi